In [1]:
import pandas as pd 
import os
import numpy as np 
from sentence_transformers import SentenceTransformer
import faiss
from transformers import AutoTokenizer
from bs4 import BeautifulSoup

c:\Users\mallo\OneDrive\Bureau\Test_SensCritique\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('interstellar_critique.csv')

print(df.columns)

Index(['id', 'URL', 'rating', 'review_date_creation',
       'review_date_last_update', 'review_hits', 'review_content',
       'review_title', 'gen_review_like_count', 'username', 'user_id'],
      dtype='object')


In [8]:
print(df.head(3))

         id                                                URL  rating  \
0  25246858  https://senscritique.com/film/xxx/critique/252...      10   
1  40035436  https://senscritique.com/film/xxx/critique/400...       4   
2  39231131  https://senscritique.com/film/xxx/critique/392...      10   

  review_date_creation review_date_last_update  review_hits  \
0       06/11/14 12:29          06/11/14 12:29        22676   
1       15/11/14 13:13          15/11/14 13:13        20186   
2       28/02/15 10:36          28/02/15 10:36        19215   

                                      review_content  \
0  Aïe Aïe Aïe, nous y voilà, Interstellar, le fi...   
1  <p>Chers lecteurs, chères lectrices,</p>\n\n<p...   
2  <p>Malgré ce que j'entends dire ou lis sur le ...   

                                        review_title  gen_review_like_count  \
0          All you need is love, love, love, love...                    513   
1  10 « bonnes » raisons relatives de ne pas aime...               

In [5]:
texte_html= df["review_title"][1] + " . " + df["review_content"][1]
print(texte_html)

10 « bonnes » raisons relatives de ne pas aimer Interstellar. . <p>Chers lecteurs, chères lectrices,</p>

<p>Avant d'entamer cette liste aussi disgracieuse que désobligeante, je tiens à préciser que je n'ai rien contre Nolan, au contraire. 
Avant de m'envoyer des menaces de mort, kidnapper un membre de ma famille ou pire encore, merci de garder votre calme et de vous contenter de cliquer sur NON à la question finale et fatidique "Et vous, avez-vous apprécié la critique ?" Ça vous fera du bien, votre honneur sera ainsi sauvé, et il y a de fortes chances que cela me pousse au suicide. Donc tout le monde y trouve son compte. 
Oui parce que ce film m'a donné envie de sauter d'un pont.</p>

<p>Sur ce, bonne lecture!</p>

<ol>
<li><p>Je vais sûrement perdre quelques abonnés à cause de cette critique.</p></li>
<li><p>Nolan a très clairement exprimé qu'il voulait rendre son film réaliste. Dans ce but, il a collaboré avec des scientifiques. C'est très bien, et ça part sûrement d'une bonne inten

In [6]:
soup = BeautifulSoup(texte_html, "html.parser")
texte_sans_html = soup.get_text()
print(texte_sans_html)

10 « bonnes » raisons relatives de ne pas aimer Interstellar. . Chers lecteurs, chères lectrices,
Avant d'entamer cette liste aussi disgracieuse que désobligeante, je tiens à préciser que je n'ai rien contre Nolan, au contraire. 
Avant de m'envoyer des menaces de mort, kidnapper un membre de ma famille ou pire encore, merci de garder votre calme et de vous contenter de cliquer sur NON à la question finale et fatidique "Et vous, avez-vous apprécié la critique ?" Ça vous fera du bien, votre honneur sera ainsi sauvé, et il y a de fortes chances que cela me pousse au suicide. Donc tout le monde y trouve son compte. 
Oui parce que ce film m'a donné envie de sauter d'un pont.
Sur ce, bonne lecture!

Je vais sûrement perdre quelques abonnés à cause de cette critique.
Nolan a très clairement exprimé qu'il voulait rendre son film réaliste. Dans ce but, il a collaboré avec des scientifiques. C'est très bien, et ça part sûrement d'une bonne intention, mais moi, petit spectateur, je dis non. Il ne

In [7]:
full_review = df["review_title"].fillna("") + " . " + df["review_content"].fillna("")

def strip_html(x):
    if not isinstance(x, str):
        return ""
    return BeautifulSoup(x, "html.parser").get_text()

df["full_review"]= full_review.map(strip_html)

In [64]:
df.columns

Index(['id', 'URL', 'rating', 'review_date_creation',
       'review_date_last_update', 'review_hits', 'review_content',
       'review_title', 'gen_review_like_count', 'username', 'user_id',
       'full_review'],
      dtype='object')

In [8]:
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
def count_tokens(text):
    return len(tokenizer.tokenize(text))

df["nb_tokens"] = df["full_review"].map(count_tokens)

Token indices sequence length is longer than the specified maximum sequence length for this model (1072 > 512). Running this sequence through the model will result in indexing errors


Certaines critiques dépassent la limite de 512 tokens du modèle, ce qui provoque une erreur.
J’applique donc un chunking pour découper les longues critiques.


In [9]:
MODEL = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
tokenizer= AutoTokenizer.from_pretrained(MODEL)

In [10]:
def chunk_by_tokens(text, tokenizer, max_len=512, stride=400):
    enc = tokenizer(
        text,
        add_special_tokens=False,
        return_attention_mask=False
    )
    ids = enc["input_ids"]

    n = len(ids)
    start = 0
    while start < n:
        end = start + max_len
        chunk_ids = ids[start:end]
        chunk_text = tokenizer.decode(chunk_ids, skip_special_tokens=True).strip()
        if chunk_text:
            yield chunk_text
        start += stride


In [18]:
def build_vector_database_for_movie(csv_path, movie_name):
    df = pd.read_csv(csv_path)
    full_review = df["review_title"].fillna("") + " . " + df["review_content"].fillna("")
    df["full_review"] = full_review.map(strip_html)
    df["movie"] = movie_name

    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    model = SentenceTransformer(MODEL)
    model.max_seq_length = 512

    all_chunks = []
    meta_rows = []

    for _, row in df.iterrows():
        text = row["full_review"]
        id = str(row["id"])
        toks = tokenizer.tokenize(text)

        if len(toks) < 512:
            all_chunks.append(text)
            meta_rows.append({
                "chunk_id": f"{id}_0",
                "id": id,
                "movie": movie_name,
                "username": row["username"],
                "full_review": row["full_review"]
            })
        else:
            for i, chunk in enumerate(chunk_by_tokens(text, tokenizer, max_len=512)):
                all_chunks.append(chunk)
                meta_rows.append({
                    "chunk_id": f"{id}_chunk{i}",
                    "id": id,
                    "movie": movie_name,
                    "username": row["username"],
                    "full_review": row["full_review"]
                })

    emb = model.encode(all_chunks, normalize_embeddings=True, show_progress_bar=True)
    
    index = faiss.IndexFlatIP(emb.shape[1])
    index.add(emb.astype("float32"))

    faiss.write_index(index, f"faiss_{movie_name}.index")
    np.save(f"embeddings_{movie_name}.npy", emb)
    pd.DataFrame(meta_rows).to_csv(f"meta_chunks_{movie_name}.csv", index=False, encoding="utf-8")


In [ ]:
def build_vector_database():
    df = pd.read_csv('interstellar_critique.csv')
    full_review = df["review_title"].fillna("") + " . " + df["review_content"].fillna("")
    df["full_review"]= full_review.map(strip_html)
    tok = AutoTokenizer.from_pretrained(MODEL)
    model = SentenceTransformer(MODEL)
    model.max_seq_length = 512

    all_chunks = []
    meta_rows = []

    for _, row in df.iterrows():
        text = row["full_review"]
        id = str(row["id"])
        toks = tokenizer.tokenize(text)
        if len(toks)< 512:
            all_chunks.append(text)
            meta_rows.append({
                "chunk_id": f"{id}_0",
                "id": id,
                "username": row["username"],
                "full_review": row["full_review"]
            })

        else:
                for i, chunk in enumerate(chunk_by_tokens(text,tokenizer,max_len=512)):
                        all_chunks.append(chunk)
                        meta_rows.append({
                        "chunk_id": f"{id}_chunk{i}",
                        "id": id,
                        "username": row["username"],
                        "full_review": row["full_review"]
                })
    emb = model.encode(all_chunks, normalize_embeddings=True, show_progress_bar=True)
    index=faiss.IndexFlatIP(emb.shape[1])
    index.add(emb.astype("float32"))
    np.save("embeddings.npy",emb)
    faiss.write_index(index, "faiss.index")
    pd.DataFrame(meta_rows).to_csv("meta_chunks.csv", index=False, encoding="utf-8")

In [111]:
if __name__ == "__main__":
    build_vector_database()

Batches: 100%|██████████| 74/74 [02:50<00:00,  2.31s/it]


In [16]:
df_fightclub= pd.read_csv("fightclub_critiques.csv")
df_fightclub.head(1)["review_content"]

0    Fight Club sort à la fin 1999 sur les écrans. ...
Name: review_content, dtype: object

In [19]:
if __name__ == "__main__":
    build_vector_database_for_movie("interstellar_critique.csv", "Interstellar")
    build_vector_database_for_movie("fightclub_critiques.csv", "Fight Club")
    

Token indices sequence length is longer than the specified maximum sequence length for this model (1072 > 512). Running this sequence through the model will result in indexing errors
Batches: 100%|██████████| 74/74 [02:36<00:00,  2.11s/it]
Token indices sequence length is longer than the specified maximum sequence length for this model (4100 > 512). Running this sequence through the model will result in indexing errors
Batches: 100%|██████████| 44/44 [00:57<00:00,  1.31s/it]


In [103]:
chunks_df= pd.read_csv('meta_chunks.csv')
index = faiss.read_index("faiss.index")
model = SentenceTransformer(MODEL)

In [104]:
chunks_df.columns
chunks_df.head(3)

,chunk_id,id,username,full_review
0,25246858_chunk0,25246858,Kobayashhi,"All you need is love, love, love, love... . Aï..."
1,25246858_chunk1,25246858,Kobayashhi,"All you need is love, love, love, love... . Aï..."
2,25246858_chunk2,25246858,Kobayashhi,"All you need is love, love, love, love... . Aï..."


In [72]:
def search_similarity(query,k=3):
    q = model.encode([query], normalize_embeddings=True)
    D,I =index.search(q.astype("float32"),k)
    resultats = []
    seen = set()

    for score , idx in zip(D[0],I[0]):
        chunk_meta= chunks_df.iloc[idx]
        orig_id = chunk_meta["id"]
        if orig_id not in seen: 
            resultats.append({
                "orig_id": orig_id,
                "full_review": chunk_meta["full_review"],
                "username": chunk_meta["username"],
                "score": float(score)
            })
            seen.add(orig_id)

        if len(resultats) >= k:
            break
    return pd.DataFrame(resultats)
        

In [73]:
print( df["full_review"][0])

All you need is love, love, love, love... . Aïe Aïe Aïe, nous y voilà, Interstellar, le film dont on ne doit pas prononcer le nom, celui qui déchaîne les passions, film de la décennie pour certains, arnaque pour d'autres. Déjà moqué pour ces symboles et idées véhiculées, décrit comme Melo larmoyant, ou encore mascarade de Science Fiction. 2001 l'Odyssey de l'Espace doit figurer à chaque paragraphe, on doit bien sûr mentionner du Tarkovski, ou des récits avec une notoriété moins évidente comme Contact ou The Black Hole et enfin faire le lien avec Gravity évidemment.

Plutôt que d'aller puiser dans des centaines de références et d'influence diverses et variées, je considère Interstellar comme le film le plus personnel de Nolan, celui dans lequel il finit par assumer un message martelé depuis toujours. Réalisateur régulièrement critiqué pour sa froideur, sa prétention ou son ambition mal placée, il est pourtant à mon sens un grand romantique méconnu. L'amour est un sentiment qui transpire

In [56]:
if __name__ == "__main__":
    example_query = """All you need is love, love, love, love... . Aïe Aïe Aïe, nous y voilà, Interstellar, le film dont on ne doit pas prononcer le nom, celui qui déchaîne les passions, film de la décennie pour certains, arnaque pour d'autres. Déjà moqué pour ces symboles et idées véhiculées, décrit comme Melo larmoyant, ou encore mascarade de Science Fiction. 2001 l'Odyssey de l'Espace doit figurer à chaque paragraphe, on doit bien sûr mentionner du Tarkovski, ou des récits avec une notoriété moins évidente comme Contact ou The Black Hole et enfin faire le lien avec Gravity évidemment.

Plutôt que d'aller puiser dans des centaines de références et d'influence diverses et variées, je considère Interstellar comme le film le plus personnel de Nolan, celui dans lequel il finit par assumer un message martelé depuis toujours. Réalisateur régulièrement critiqué pour sa froideur, sa prétention ou son ambition mal placée, il est pourtant à mon sens un grand romantique méconnu. L'amour est un sentiment qui transpire chaque parcelle de ses films, Mémento réputé pour sa narration déstructurée et jouant avec la mémoire à court terme de son héros est avant tout l'histoire d'un homme à la recherche du meurtrier de sa femme, obsession découlant d'un amour perdu. Le prestige plus connu pour son utilisation quasi surnaturelle de la magie, parle avant tout d'une rivalité entre deux hommes submergés par des motivations équivalentes toujours générées par ce même sentiment universel, l'un ayant perdu sa femme, l'autre voulant protéger sa fille. Même chez Batman, c'est bien l'amour pour une femme qui sera le moteur de départ pour créer ce justicier noir ou au contraire permettre la naissance de l'un des Bad Guy les plus réputés du comics. Quant à Inception au milieu des différents degrés de rêves, des théories qui se sont abattues partout dans le monde, de cette foutue toupie, s'agit t-il d'autre chose que de la souffrance d'un Homme prêt à tout pour profiter de souvenirs entretenus avec sa femme quitte à perdre pied avec la réalité.

Les rêves, la mémoire, les super héros, et maintenant le voyage spatial. Tout ceci n'est qu'un gigantesque décor servant de prétexte dans lequel les artifices en tous genres apporteront une atmosphère peut être trop guindée parfois, mais en tout cas présent avec un parti pris assumé et cette patte immédiatement identifiable.

Et c'est avec cette forme toujours plus excentrique, plus prestigieuse, avec cette poudre aux yeux qu'un fond qui pourrait sentir le rose bonbon chez plus d'un fonctionne ici. On reprend un schéma qui fonctionne, une vraie gueule pour porter son récit, et c'est presque naturellement que le fraîchement Oscarisé McConaughey s'y colle avec brio. Jamais dans l'excès toujours dans le questionnement philosophique certes mais aux antipodes de son nihiliste chez True Détective, campe ici un humaniste bien plus crédible en ingénieur que Marky Mark chez Bay. Omniprésent sous la caméra de Nolan, c'est à travers son regard que l'on vivra les aventures les plus périlleuses, les plus épiques ou les plus émouvantes, le tout sous fond de thèmes bien plus inspirés chez Hans Zimmer que l'énorme copié collé que l'on doit supporter depuis des années. 

Avant d'être un pur film de science fiction comme on l'entend et comme on peut l'attendre, Interstellar est un film sur le sentiment le plus universel, celui qui sert de moteur, qui donne de l'importance à nos vies qui semblent si infimes au vu de l'étendu de l'univers. En découle une ode au voyage, à l'espoir, ou à la connaissance, certes teinté d'imperfections, de petites facilités scénaristiques par moment, d'enchaînements un peu flous mais aisément annihiler par une maîtrise et par une cohérence dans une filmographie qui m'ont offert exactement le spectacle que j'attendais. Une fable complexe en apparence élaborant tout un tas de théories scientifique, le temps, le voyage spatial, le sens de la vie uniquement là pour épauler son propos initial, la classe.

Ps : 300 critiques déjà, ça en fait des conneries écrites dis donc, merci à tous pour vos commentaires réguliers et à bientôt enfin pas trop non plus j'espère pour les 1000 !"""
    res = search_similarity(example_query, k=3)
    print(res[["orig_id", "score", "username", "full_review"]])

     orig_id     score       username  \
0  110037233  0.772847         Andros   
1   40813793  0.718422  MentheFraiche   
2   40824515  0.717366       Pipotroi   

                                         full_review  
0  InterstellAr avec un grand A ;) . " L'amour es...  
1  Lyrique… . S’il avait été écrit, Interstellar ...  
2  Interstellar, à la hauteur de ses ambitions ? ...  


j’encode directement la critique complète et je cherche les 3 plus proches dans FAISS. 
Même si la critique existe déjà dans le CSV, le modèle renvoie une autre avec un score ≈ 0.77, car en fait la recherche se fait sur les chunks présents dans la base, pas sur les textes entiers. C'est pourquoi j'ai pensé à chunker la requete aussi.

In [105]:
def embed_query(query_text, model, tokenizer):
   
    toks = tokenizer.tokenize(query_text)
    if len(toks) < 512:
        vec = model.encode([query_text], normalize_embeddings=True)
        return vec.astype("float32")

    parts = list(chunk_by_tokens(query_text, tokenizer, max_len=512))
    part_vecs = model.encode(parts, normalize_embeddings=True)
    mean_vec = np.mean(part_vecs, axis=0, keepdims=True)
    return mean_vec.astype("float32")

In [79]:
def search_similarity_chunking(query_text, k, chunks_df, index, model, tokenizer):
   
    q_vec = embed_query(query_text, model, tokenizer)
    D, I = index.search(q_vec, k)

    resultats = []
    seen = set()

    for score, idx in zip(D[0], I[0]):
        chunk_meta = chunks_df.iloc[idx]
        orig_id = chunk_meta["id"]

        if orig_id not in seen:
            resultats.append({
                "orig_id": orig_id,
                "full_review": chunk_meta["full_review"],
                "username": chunk_meta["username"],
                "score": float(score)
            })
            seen.add(orig_id)

        if len(resultats) >= k:
            break

    return pd.DataFrame(resultats)


In [82]:
if __name__ == "__main__":
    example_query = """All you need is love, love, love, love... . Aïe Aïe Aïe, nous y voilà, Interstellar, le film dont on ne doit pas prononcer le nom, celui qui déchaîne les passions, film de la décennie pour certains, arnaque pour d'autres. Déjà moqué pour ces symboles et idées véhiculées, décrit comme Melo larmoyant, ou encore mascarade de Science Fiction. 2001 l'Odyssey de l'Espace doit figurer à chaque paragraphe, on doit bien sûr mentionner du Tarkovski, ou des récits avec une notoriété moins évidente comme Contact ou The Black Hole et enfin faire le lien avec Gravity évidemment.

Plutôt que d'aller puiser dans des centaines de références et d'influence diverses et variées, je considère Interstellar comme le film le plus personnel de Nolan, celui dans lequel il finit par assumer un message martelé depuis toujours. Réalisateur régulièrement critiqué pour sa froideur, sa prétention ou son ambition mal placée, il est pourtant à mon sens un grand romantique méconnu. L'amour est un sentiment qui transpire chaque parcelle de ses films, Mémento réputé pour sa narration déstructurée et jouant avec la mémoire à court terme de son héros est avant tout l'histoire d'un homme à la recherche du meurtrier de sa femme, obsession découlant d'un amour perdu. Le prestige plus connu pour son utilisation quasi surnaturelle de la magie, parle avant tout d'une rivalité entre deux hommes submergés par des motivations équivalentes toujours générées par ce même sentiment universel, l'un ayant perdu sa femme, l'autre voulant protéger sa fille. Même chez Batman, c'est bien l'amour pour une femme qui sera le moteur de départ pour créer ce justicier noir ou au contraire permettre la naissance de l'un des Bad Guy les plus réputés du comics. Quant à Inception au milieu des différents degrés de rêves, des théories qui se sont abattues partout dans le monde, de cette foutue toupie, s'agit t-il d'autre chose que de la souffrance d'un Homme prêt à tout pour profiter de souvenirs entretenus avec sa femme quitte à perdre pied avec la réalité.

Les rêves, la mémoire, les super héros, et maintenant le voyage spatial. Tout ceci n'est qu'un gigantesque décor servant de prétexte dans lequel les artifices en tous genres apporteront une atmosphère peut être trop guindée parfois, mais en tout cas présent avec un parti pris assumé et cette patte immédiatement identifiable.

Et c'est avec cette forme toujours plus excentrique, plus prestigieuse, avec cette poudre aux yeux qu'un fond qui pourrait sentir le rose bonbon chez plus d'un fonctionne ici. On reprend un schéma qui fonctionne, une vraie gueule pour porter son récit, et c'est presque naturellement que le fraîchement Oscarisé McConaughey s'y colle avec brio. Jamais dans l'excès toujours dans le questionnement philosophique certes mais aux antipodes de son nihiliste chez True Détective, campe ici un humaniste bien plus crédible en ingénieur que Marky Mark chez Bay. Omniprésent sous la caméra de Nolan, c'est à travers son regard que l'on vivra les aventures les plus périlleuses, les plus épiques ou les plus émouvantes, le tout sous fond de thèmes bien plus inspirés chez Hans Zimmer que l'énorme copié collé que l'on doit supporter depuis des années. 

Avant d'être un pur film de science fiction comme on l'entend et comme on peut l'attendre, Interstellar est un film sur le sentiment le plus universel, celui qui sert de moteur, qui donne de l'importance à nos vies qui semblent si infimes au vu de l'étendu de l'univers. En découle une ode au voyage, à l'espoir, ou à la connaissance, certes teinté d'imperfections, de petites facilités scénaristiques par moment, d'enchaînements un peu flous mais aisément annihiler par une maîtrise et par une cohérence dans une filmographie qui m'ont offert exactement le spectacle que j'attendais. Une fable complexe en apparence élaborant tout un tas de théories scientifique, le temps, le voyage spatial, le sens de la vie uniquement là pour épauler son propos initial, la classe.

Ps : 300 critiques déjà, ça en fait des conneries écrites dis donc, merci à tous pour vos commentaires réguliers et à bientôt enfin pas trop non plus j'espère pour les 1000 !"""
    res = search_similarity_chunking(example_query, 3, chunks_df, index, model, tokenizer)
    print(res[["orig_id", "score", "username", "full_review"]])

    orig_id     score           username  \
0  41339127  0.555459        LaLuneMauve   
1  45734343  0.548838            Benreal   
2  42028110  0.540186  Djack-le-Flemmard   

                                         full_review  
0  Étourdissant . « Interstellar », film de 2014 ...  
1  Un pillier est posé, une référence est créé ! ...  
2  Un monde de poussière, où Swiffer règne en Maî...  


pour éviter l’erreur sur les textes longs,j'ai découpé la requête en chunks de 512 tokens et j'ai fait la moyenne
des vecteurs. Ça marche sans planter, mais la moyenne rend les résultats moins précis.

Donc au lieu de moyenner, je cherche séparément pour chaque chunk, puis je regroupe les meilleurs résultats. Cette méthode 
gère les longs textes tout en retrouvant mieux les critiques similaires.

In [106]:
def search_similarity_3(query_text, k, chunks_df, index, model, tokenizer):
    toks = tokenizer.tokenize(query_text)

    if len(toks) <= 512:
        q_vec = model.encode([query_text], normalize_embeddings=True).astype("float32")
        D, I = index.search(q_vec, k)

        candidates = [(float(score), int(idx)) for score, idx in zip(D[0], I[0])]
    else:
        # chercher chunk par chunk
        parts = list(chunk_by_tokens(query_text, tokenizer, max_len=512))
        part_vecs = model.encode(parts, normalize_embeddings=True).astype("float32")

        # on agrège tous les résultats des chunks
        candidates = []
        for vec in part_vecs:
            vec = vec.reshape(1, -1) 
            D, I = index.search(vec, k)
            for score, idx in zip(D[0], I[0]):
                candidates.append((float(score), int(idx)))

        # Tri par score desc
        candidates.sort(key=lambda x: x[0], reverse=True)

    results = []
    seen = set()
    for score, idx in candidates:
        chunk_meta = chunks_df.iloc[idx]
        orig_id = chunk_meta["id"]
        if orig_id in seen:
            continue

        results.append({
            "orig_id": orig_id,
            "full_review": chunk_meta["full_review"],
            "username": chunk_meta["username"],
            "score": float(score)
        })
        seen.add(orig_id)

        if len(results) >= k:
            break

    return pd.DataFrame(results)


In [ ]:
if __name__ == "__main__":
    example_query = """All you need is love, love, love, love... . Aïe Aïe Aïe, nous y voilà, Interstellar, le film dont on ne doit pas prononcer le nom, celui qui déchaîne les passions, film de la décennie pour certains, arnaque pour d'autres. Déjà moqué pour ces symboles et idées véhiculées, décrit comme Melo larmoyant, ou encore mascarade de Science Fiction. 2001 l'Odyssey de l'Espace doit figurer à chaque paragraphe, on doit bien sûr mentionner du Tarkovski, ou des récits avec une notoriété moins évidente comme Contact ou The Black Hole et enfin faire le lien avec Gravity évidemment.

Plutôt que d'aller puiser dans des centaines de références et d'influence diverses et variées, je considère Interstellar comme le film le plus personnel de Nolan, celui dans lequel il finit par assumer un message martelé depuis toujours. Réalisateur régulièrement critiqué pour sa froideur, sa prétention ou son ambition mal placée, il est pourtant à mon sens un grand romantique méconnu. L'amour est un sentiment qui transpire chaque parcelle de ses films, Mémento réputé pour sa narration déstructurée et jouant avec la mémoire à court terme de son héros est avant tout l'histoire d'un homme à la recherche du meurtrier de sa femme, obsession découlant d'un amour perdu. Le prestige plus connu pour son utilisation quasi surnaturelle de la magie, parle avant tout d'une rivalité entre deux hommes submergés par des motivations équivalentes toujours générées par ce même sentiment universel, l'un ayant perdu sa femme, l'autre voulant protéger sa fille. Même chez Batman, c'est bien l'amour pour une femme qui sera le moteur de départ pour créer ce justicier noir ou au contraire permettre la naissance de l'un des Bad Guy les plus réputés du comics. Quant à Inception au milieu des différents degrés de rêves, des théories qui se sont abattues partout dans le monde, de cette foutue toupie, s'agit t-il d'autre chose que de la souffrance d'un Homme prêt à tout pour profiter de souvenirs entretenus avec sa femme quitte à perdre pied avec la réalité.

Les rêves, la mémoire, les super héros, et maintenant le voyage spatial. Tout ceci n'est qu'un gigantesque décor servant de prétexte dans lequel les artifices en tous genres apporteront une atmosphère peut être trop guindée parfois, mais en tout cas présent avec un parti pris assumé et cette patte immédiatement identifiable.

Et c'est avec cette forme toujours plus excentrique, plus prestigieuse, avec cette poudre aux yeux qu'un fond qui pourrait sentir le rose bonbon chez plus d'un fonctionne ici. On reprend un schéma qui fonctionne, une vraie gueule pour porter son récit, et c'est presque naturellement que le fraîchement Oscarisé McConaughey s'y colle avec brio. Jamais dans l'excès toujours dans le questionnement philosophique certes mais aux antipodes de son nihiliste chez True Détective, campe ici un humaniste bien plus crédible en ingénieur que Marky Mark chez Bay. Omniprésent sous la caméra de Nolan, c'est à travers son regard que l'on vivra les aventures les plus périlleuses, les plus épiques ou les plus émouvantes, le tout sous fond de thèmes bien plus inspirés chez Hans Zimmer que l'énorme copié collé que l'on doit supporter depuis des années. 

Avant d'être un pur film de science fiction comme on l'entend et comme on peut l'attendre, Interstellar est un film sur le sentiment le plus universel, celui qui sert de moteur, qui donne de l'importance à nos vies qui semblent si infimes au vu de l'étendu de l'univers. En découle une ode au voyage, à l'espoir, ou à la connaissance, certes teinté d'imperfections, de petites facilités scénaristiques par moment, d'enchaînements un peu flous mais aisément annihiler par une maîtrise et par une cohérence dans une filmographie qui m'ont offert exactement le spectacle que j'attendais. Une fable complexe en apparence élaborant tout un tas de théories scientifique, le temps, le voyage spatial, le sens de la vie uniquement là pour épauler son propos initial, la classe.

Ps : 300 critiques déjà, ça en fait des conneries écrites dis donc, merci à tous pour vos commentaires réguliers et à bientôt enfin pas trop non plus j'espère pour les 1000 !"""
    res = search_similarity_3(example_query, 3, chunks_df, index, model, tokenizer)
    print(res[["orig_id", "score", "username", "full_review"]])

     orig_id     score      username  \
0   25246858  1.000000    Kobayashhi   
1  110037233  0.772847        Andros   
2   41687122  0.740835  JanosValuska   

                                         full_review  
0  All you need is love, love, love, love... . Aï...  
1  InterstellAr avec un grand A ;) . " L'amour es...  
2  Coming home. . J’ai trouvé ça immense. Je ne s...  


Aide IA

J’ai utilisé ChatGPT pour la fonction de chunking , pour reformuler certains commentaires explicatifs, pour mettre directement le diagramme d'architecture dans github et structurer le README.